# Chapter 6
# Bayesian optimization: Automate continuous parameter tuning

In [ ]:
import numpy as np
import scipy
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt
from e4e import E4E

e4e = E4E(chapter=6)

# 6.1 Optimize a single compiler parameter, a visual explanation

In [ ]:
# Listing 6.1 JIT CPU time simulator
def jit_plus_server(parameters):
    x = np.array(parameters)
    d = len(x)
    x1 = x - 0.15*np.ones(shape=(d,))
    x2 = x - 0.85*np.ones(shape=(d,))
    cpu_time = 2 - np.exp(-10*x1**2) - 0.5*np.exp(-10*x2**2)
    return cpu_time.mean() + .005*np.random.normal()

### 6.1.2	Run the initial experiment

In [ ]:
np.random.seed(17)
jit_plus_server([0.5])

### 6.1.3	Analyze: Model the response surface

In [ ]:
class GPR_viz: # same as GPR4
    def __init__(self, parameters, measurements, sigma):        
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma
        
        self.mean_y = self.y.mean()        
        if len(self.y) > 1:
            self.std_y = self.y.std()
        else:
            self.std_y = 1
            
        self.y -= self.mean_y
        
    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        kernels_x_query = np.array([
            self.kernel(x, query_parameter)
            for x in self.x
        ])
        kernels_x_x = np.array([
            [
                self.kernel(x1, x2)
                for x1 in self.x
            ]
            for x2 in self.x
        ])

        weights = kernels_x_query.T @ np.linalg.pinv(kernels_x_x)
        expectation = self.mean_y + weights @ self.y
        uncertainty_squared = 1 - weights @ kernels_x_query
        return expectation, self.std_y*np.sqrt(np.maximum(0, uncertainty_squared))

In [ ]:
def plot_example_gpr(GPR, ax, x, y, err_bars=False, bottom_trace=False):
    x = np.array(x)
    y = np.array(y)
    gpr = GPR(x, y, sigma=0.15)
    x_hats = np.linspace(0,1,100)
    y_hats = []
    sigma_y_hats = []
    for x_hat in x_hats:
        ret = gpr.estimate(x_hat)
        try:
            y_hat, sigma_y_hat = ret
        except:
            y_hat = ret
            sigma_y_hat = 0
        y_hats.append(y_hat)
        sigma_y_hats.append(sigma_y_hat)

    y_hats = np.array(y_hats)
    sigma_y_hats = np.array(sigma_y_hats)
    
    if err_bars:
        ax.fill_between(x_hats.flatten(),
                 y_hats - sigma_y_hats,
                 y_hats + sigma_y_hats,
                 color=e4e.color_4,
                 # alpha=alpha_err,
                 linewidth=1);

        
    ax.plot(x, y, 'o', color=e4e.color_1, markersize=5);
    ax.plot(x_hats, y_hats, ':', color=e4e.color_2);
    
    if bottom_trace:
        y_bots = y_hats - sigma_y_hats
        ax.plot(x_hats, y_bots, color=e4e.color_1, linewidth=2);
    
    # i = np.where(y_hat == y_hat.min())[0]
    
    # ax.axis([-.05, 1.05, -.1, 2])
    # ax.set_xticklabels([])
    # ax.set_yticklabels([])

In [ ]:
np.random.seed(17)
parameter0 = 0.5
cpu_time0 = jit_plus_server([parameter0])

print (parameter0, cpu_time0)

ax = plt.gca()
plot_example_gpr(GPR_viz, ax, [parameter0], [cpu_time0], err_bars=True)
plt.xlabel('parameter')
plt.ylabel('CPU time')
e4e.save_fig(3)

### 6.1.4	Design: Select the parameter value to measure next

In [ ]:
np.random.seed(17)
parameter1 = 0
cpu_time1 = jit_plus_server([parameter1])
print (cpu_time1)

In [ ]:
ax = plt.gca()
plot_example_gpr(
    GPR_viz,
    ax,
    np.array([parameter0, parameter1]),
    np.array([cpu_time0, cpu_time1]),
    err_bars=True
)
plt.xlabel('parameter')
plt.ylabel('CPU time')
e4e.save_fig(4)

### 6.1.5	Design: Balance exploration with exploitation

In [ ]:
ax = plt.gca()
plot_example_gpr(
    GPR_viz,
    ax,
    np.array([parameter0, parameter1]),
    np.array([cpu_time0, cpu_time1]),

    err_bars=True,
)
plt.xlabel('parameter')
plt.ylabel('CPU time')

e4e.vertical_line(0)
e4e.vertical_line(1)


plt.annotate("minimize\nCPU time", xy=[0, 1.47],
             xytext=[.05, 1.5],
             arrowprops=e4e.arrow_props
            )

plt.annotate("maximize\nmodel\nuncertainty", xy=[1, 1.27],
             xytext=[.7, 1.2],
             arrowprops=e4e.arrow_props
            )

e4e.save_fig(6)

In [ ]:
ax = plt.gca()
plot_example_gpr(
    GPR_viz,
    ax,
    np.array([.5, 0]),
    np.array([1.56, 1.20]),
    err_bars=True,
    bottom_trace=True
)
plt.xlabel('parameter')
plt.ylabel('CPU time')


e4e.vertical_line(.1111111)

plt.annotate("balance\nCPU time and\nmodel uncertainty", xy=[.111111 + .005, 1.35],
             xytext=[.4, 1.2],
             arrowprops=e4e.arrow_props
            )
e4e.save_fig(7)

In [ ]:
parameters = np.array([0.5, 0, 0.111, .2020, 1.0, .1515])
measurements = [1.56, 1.20, 1.0036, 1.02231, 1.598, .99696]
gpr = GPR_viz(parameters, measurements, sigma=.15)
x_hats = np.linspace(0,1,100)
y_hats = np.array([gpr.estimate(x_hat) for x_hat in x_hats])
lcb = y_hats[:,0] - y_hats[:,1]
i = np.where(lcb == lcb.min())[0]
print (x_hats[i], jit_plus_server(x_hats[i]))

In [ ]:

fig, ((ax11, ax12), (ax21, ax22)) = plt.subplots(2,2)

plot_example_gpr(
    GPR_viz,
    ax11,
    [.5, 0, .11],
    [1.56, 1.20, 1.0036],
    err_bars=True,
    bottom_trace=True
)

plot_example_gpr(
        GPR_viz,
    ax12,
    [.5, 0, 0.111, .2020],
    [1.56, 1.20, 1.0036, 1.02231],
    err_bars=True,
    bottom_trace=True
)

plot_example_gpr(
        GPR_viz,
    ax21,
    [.5, 0, 0.111, .2020, 1.0],
    [1.56, 1.20, 1.0036, 1.02231, 1.598],
    err_bars=True,
    bottom_trace=True
)

plot_example_gpr(
        GPR_viz,
    ax22,
    [.5, 0, 0.111, .2020, 1.0, .1515],
    [1.56, 1.20, 1.0036, 1.02231, 1.598, .99696],
    err_bars=True,
    bottom_trace=True
)

c = list(ax22.axis())
c[2] -= .1
ax11.axis(c)
ax12.axis(c)
ax21.axis(c)
ax22.axis(c)


ax11.text(0,1.55,'(a)')
ax12.text(0,1.55,'(b)')
ax21.text(0,1.55,'(c)')
ax22.text(0,1.55,'(d)')



e4e.save_fig(8)

# 6.2	Model the response surface with gaussian process regression

In [ ]:
(1.54 + 1.21)/2 

In [ ]:
ax = plt.gca()
plot_example_gpr(
    GPR_viz,
    ax,
    [.5, 0],
    [1.56, 1.21],
    err_bars=True
)
plt.xlabel('parameter')
plt.ylabel('CPU time')
plt.legend(['_nolegend_', 'expected CPU time', 'uncertainty in estimate'])
e4e.save_fig(9)

In [ ]:
.8*1.52 + .2*1.21

In [ ]:
50*1.52 + 50*1.21

### 6.2.1	Estimate the expected CPU time

In [ ]:
# Listing 6.2 Beginnings of gaussian process regression
class GPR1:
    def __init__(self, parameters, measurements):        
        self.x = parameters
        self.y = np.array(measurements)
        self.mean_y = self.y.mean()
        
    def estimate(self, query_parameter):
        return self.mean_y

#### WEIGHT NEARER MEASUREMENTS MORE

In [ ]:
d = np.linspace(0, 1, 100)
w = np.exp(-d**2 / .1)
plt.plot(d, w, '--', color=e4e.color_1);
plt.xlabel('distance = np.abs(x - parameter)')
plt.ylabel('weight')
e4e.save_fig(10)

In [ ]:
# Listing 6.3 GPR with a weighted average
class GPR2:
    def __init__(self, parameters, measurements, sigma):        
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma
        
        self.mean_y = self.y.mean()
        self.y -= self.mean_y
        
    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        weights = [
            self.kernel(x, query_parameter)
            for x in self.x
        ]
        weights = np.array(weights)
        weights = weights / weights.sum()
        return self.mean_y + weights @ self.y

In [ ]:
parameters = np.array([0.5, 0.0])
measurements = [1.52, 1.21]
gpr2 = GPR2(parameters, measurements, sigma=0.25)
print (gpr2.estimate(0.25))
print (gpr2.estimate(0.4))

#### DON’T OVERWEIGHT CLUSTERED MEASUREMENTS

In [ ]:
parameters = np.array([0.5, 0.0, 0.4])
measurements = [1.52, 1.21, gpr2.estimate(0.4)]
gpr2a = GPR2(parameters, measurements, sigma=0.25)
print (gpr2a.estimate(0.25))
plt.plot(parameters, measurements, 'o', color=e4e.color_1);
plt.plot([0.25], [gpr2a.estimate(0.25)], '^', color=e4e.color_2)
plt.plot([0.25], [gpr2.estimate(0.25)], 'x', color=e4e.color_2)
plt.legend(['measurements', 'distance weights', 'distance weights\nand cluster down-weighting'])
e4e.save_fig(11)

In [ ]:
# Listing 6.4 Full gaussian process regression
class GPR3:
    def __init__(self, parameters, measurements, sigma):        
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma
        
        self.mean_y = self.y.mean()
        self.y -= self.mean_y
        
    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        kernels_x_query = np.array([
            self.kernel(x, query_parameter)
            for x in self.x
        ])
        kernels_x_x = np.array([
            [
                self.kernel(x1, x2)
                for x1 in self.x
            ]
            for x2 in self.x
        ])

        weights = kernels_x_query.T @ np.linalg.inv(kernels_x_x)
        return self.mean_y + weights @ self.y

In [ ]:
parameters = np.array([0.5, 0.0])
measurements = [1.52, 1.21]

gpr3 = GPR3(parameters, measurements, sigma=0.15)
x_hats = np.linspace(0,1,100)
y_hats = [gpr3.estimate(x_hat) for x_hat in x_hats]

plt.plot(parameters, measurements, 'o', color=e4e.color_1, markersize=5);
plt.plot(x_hats, y_hats, ':', color=e4e.color_2);
plt.xlabel('parameter')
plt.ylabel('CPU time')
e4e.save_fig(12)

In [ ]:
def plot_gpr_ex(GPR, err_bars):
    fig, ax = plt.subplots(2,2)

    np.random.seed(18)
    x = np.random.uniform(size=(3,))
    y = 1 - (x-0.4)**2
    plot_example_gpr(GPR, ax[0][0], x, y, err_bars)

    x = np.random.uniform(size=(4,))
    y = (x-0.7)**2 + 2*x**3
    plot_example_gpr(GPR, ax[0][1], x, y, err_bars)

    x = np.random.uniform(size=(4,))
    y = 1 - (x-0.3)**2 + .25*np.sin(10*x)
    plot_example_gpr(GPR, ax[1][0], x, y, err_bars)

    x = np.random.uniform(size=(5,))
    y = 1 - (x-0.9)**2 + .1*np.sin(10*(x-0.2))
    plot_example_gpr(GPR, ax[1][1], x, y, err_bars)

    ax[1][1].legend(['measurements', 'GPR estimate'], loc='lower right')
    ax[1][0].set_xlabel('parameter')
    ax[1][1].set_xlabel('parameter')
    ax[1][0].set_ylabel('business metric')
    ax[0][0].set_ylabel('business metric')

    if err_bars:
        e4e.save_fig(16)
    else:
        e4e.save_fig(13)

In [ ]:
plot_gpr_ex(GPR3, err_bars=False)

#### COMPARE TO RSM

#### REVIEW

### 6.2.2	Estimate uncertainty with GPR

In [ ]:
d = np.linspace(0, 1, 100)
w = np.exp(-d**2 / 0.1)
plt.plot(d, w, '--', color=e4e.color_1);
plt.xlabel('distance from a measurement')
plt.ylabel('certainty in estimate')
e4e.save_fig(14)

In [ ]:
# Listing 6.5 Complete gaussian process regression
class GPR4:
    def __init__(self, parameters, measurements, sigma):        
        self.x = parameters
        self.y = np.array(measurements)
        self.sigma = sigma
        
        self.mean_y = self.y.mean()        
        if len(self.y) > 1:
            self.std_y = self.y.std()
        else:
            self.std_y = 1
            
        self.y -= self.mean_y
        
    def kernel(self, x1, x2):
        distance_squared = ((x1-x2)**2).sum()
        return np.exp( -distance_squared/(2*self.sigma**2) )

    def estimate(self, query_parameter):
        kernels_x_query = np.array([
            self.kernel(x, query_parameter)
            for x in self.x
        ])
        kernels_x_x = np.array([
            [
                self.kernel(x1, x2)
                for x1 in self.x
            ]
            for x2 in self.x
        ])

        weights = kernels_x_query.T @ np.linalg.pinv(kernels_x_x)
        expectation = self.mean_y + weights @ self.y
        uncertainty_squared = 1 - weights @ kernels_x_query
        uncertainty = np.sqrt(uncertainty_squared)
        return expectation, self.std_y*uncertainty

In [ ]:
parameters = np.array([0.5, 0.0])
measurements = [1.52, 1.21]

ax = plt.gca()
plot_example_gpr(
    GPR4,
    ax,
    parameters,
    measurements,
    err_bars=True
)

ax.set_xlabel('parameter')
ax.set_ylabel('CPU time')
ax.legend(['_nolegend_', 'expected CPU time', 'uncertainty in estimate'])


e4e.save_fig(15)

In [ ]:
plot_gpr_ex(GPR4, err_bars=True)

## 6.3	Optimize over an acquisition function

In [ ]:
parameters = [0.5, 0.0]
measurements = [1.52, 1.21]


ax = plt.gca()
plot_example_gpr(
    GPR_viz,
    ax,
    parameters,
    measurements,
    err_bars=True,
    bottom_trace=True
)
plt.xlabel('parameter')
plt.ylabel('CPU time')


e4e.vertical_line(.111111)

plt.annotate("design of next experiment", xy=[0.111111 + 0.005, 1.35],
             xytext=[0.4, 1.2],
             arrowprops=e4e.arrow_props
            )
e4e.save_fig(18)

### 6.3.1	Minimize the acquisition function

In [ ]:
parameters = np.array([0.5, 0.0])
measurements = [1.52, 1.21]

gpr4 = GPR4(parameters, measurements, sigma=0.15)
x_hats = np.linspace(0,1,100)
y_hats, sigma_y_hats = zip(*[gpr4.estimate(x_hat) for x_hat in x_hats])
k = 1
lcb = np.array(y_hats) - k*np.array(sigma_y_hats)
i = np.where(lcb == lcb.min())
print (x_hats[i])

## 6.4	Optimize all seven compiler parameters

### 6.4.1	Random search

In [ ]:
# Listing 6.6 Random search
def evaluate(gpr, x):
    x = np.mod(x, 1)
    y, sigma_y = gpr.estimate(x)
    lcb = y - sigma_y
    return x, lcb
    
def random_search(gpr, num_parameters, num_iterations=1000):
    step_size = 0.1
    x_current = np.random.normal(size=num_parameters)
    x_current, lcb_current = evaluate(gpr, x_current)
    trace = []
    for _ in range(num_iterations):
        x_test = (
            x_current
            + step_size*np.random.normal(size=num_parameters)
        )
        x_test, lcb_test = evaluate(gpr, x_test)
        if lcb_test < lcb_current:
            lcb_current = lcb_test
            x_current = x_test
        trace.append( lcb_current )
    return x_current, np.array(trace)

In [ ]:
np.random.seed(17)
parameters = [ np.array([0.5]), np.array([0.0]) ]
measurements = [1.52, 1.21]

gpr4 = GPR4(parameters, measurements, sigma=0.15)
x_opt, trace = random_search(gpr4, 1)
print (x_opt)

In [ ]:
plt.plot(trace, '.--', color=e4e.color_1);
plt.xlabel('iteration')
plt.ylabel('lcb_current')
e4e.save_fig(18)

### 6.4.2	A complete Bayesian optimization

In [ ]:
# Listing 6.7 Bayesian optimizer
class BayesianOptimizer:
    def __init__(self, num_parameters):
        self.num_parameters = num_parameters
        self.parameters = []
        self.measurements = []
        self.x0 = np.array([0.5]*num_parameters)
            
    def ask(self):
        if len(self.measurements)==0:
            return self.x0
        return self.new_parameter()
        
    def new_parameter(self):
        gpr = GPR4(
            self.parameters,
            self.measurements,
            sigma=0.15,
        )
        return random_search(gpr, self.num_parameters)[0]

    def tell(self, parameter, measurement):
        self.parameters.append(parameter)
        self.measurements.append(measurement)

In [ ]:
np.random.seed(7)
bo = BayesianOptimizer(num_parameters=7)
trace = []
cpu_time = None
i = 0

In [ ]:
bo = BayesianOptimizer(num_parameters=7)
trace = []
for _ in range(48):
    parameter = bo.ask()
    cpu_time = jit_plus_server(parameter)
    bo.tell(parameter, cpu_time)
    trace.append(cpu_time)

In [ ]:
plt.plot(trace, '.--', color=e4e.color_1);
plt.xlabel('iteration')
plt.ylabel('measured CPU time')
e4e.save_fig(20)

# 6.5	Summary